<span style="color:red">WIP. Markdown tips:  https://www.markdownguide.org/basic-syntax/#</span>

<h1><center> Lecture notes</center></h1>


# A. KIVA model

In [1]:
import math
from PyCh import *


PyChi version 20210927 imported succesfully.
 


In [2]:

# =================================
# # Pod definition
# =================================
@dataclass
class Pod:
    id: int = 0

# =================================
# # Generator definition
# =================================
def Generator(env, a, N):
    for i in range(N):
        x = Pod()
        send = a.send(x)
        yield env.execute(send)

# =================================
# # Storage definition
# =================================
def Storage(env, a, b, la):
    while True:
        receive = a.receive()
        x = yield env.execute(receive)

        delay = random.exponential(1.0 / la)
        yield env.timeout(delay)

        send = b.send(x)
        yield env.execute(send)

# =================================
# # Buffer definition
# =================================
def Buffer(env, a, b):
    xs = [] # list of pods
    while True:
        receiving = a.receive()
        sending = b.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]


# =================================
# # Pick definition
# =================================
def Pick(env, a, b, mu, n):
    for i in range(n):
        receive = a.receive()
        x = yield env.execute(receive)

        delay = random.exponential(1.0 / mu)
        yield env.timeout(delay)

        send = b.send(x)
        yield env.execute(send)
    print("TH = %g" % ( n / env.now))

# =================================
# # KIVA Model
# =================================
N = 1       # Number of robots
la = 4.0    # average rate / hour for storing and retrieving racks (averages to 15 min per task)
mu = 20.0   # average rate / hour for picking of racks (averages to 3 min per task)

env = Environment()
a = Channel(env)
b = Channel(env)
c = Channel(env)

G = env.process(Generator(env, a, N))
Ss = [env.process(Storage(env, a, b, la)) for j in range(N)]
B = env.process(Buffer(env, b, c))
P = env.process(Pick(env, c, a, mu, 10000))

env.run()
print ("simulation has ended")

TH = 3.33494
simulation has ended



# B. Zero-buffer model

In [12]:

# =================================
# # Job definition
# =================================
@dataclass
class Job:
    id: int = 0

# =================================
# # Generator definition
# =================================
def Generator(env, a, u):
    i = 0
    while True:
        x = Job(id = i)
        send = a.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)
        i = i+1

# =================================
# # Machine definition
# =================================
def Machine(env, a, b, u):
    while True:
        receive = a.receive()
        x = yield env.execute(receive)

        delay = u()
        yield env.timeout(delay)

        send = b.send(x)
        yield env.execute(send)

# =================================
# # Exit definition
# =================================
def Exit(env, a, n):
    x = Job()
    while x.id < n:
        receive = a.receive()
        x = yield env.execute(receive)
    print("TH = %g" % ( x.id / env.now))

# =================================
# # GME Model
# =================================
ta = 1.0
te = 1.0
n  = 1000

env = Environment()
a = Channel(env)
b = Channel(env)

G = env.process(Generator(env, a, lambda: ta))
M = env.process(Machine(env, a, b, lambda: te))
E = env.process(Exit(env, b, n))

env.run()
print ("simulation has ended")

TH = 0.9999
simulation has ended


# C. Finite Buffer Model

In [13]:

# =================================
# # Job definition
# =================================
@dataclass
class Job:
    id: int = 0

# =================================
# # Generator definition
# =================================
def Generator(env, a, u):
    i = 0
    while True:
        x = Job(id = i)
        send = a.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)
        i = i+1

# =================================
# # Buffer definition
# =================================
def Buffer(env, a, b, N):
    xs = []
    while True:
        receiving = a.receive() if len(xs)<N else None
        sending = b.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]       
        
# =================================
# # Machine definition
# =================================
def Machine(env, a, b, u):
    while True:
        receive = a.receive()
        x = yield env.execute(receive)

        delay = u()
        yield env.timeout(delay)

        send = b.send(x)
        yield env.execute(send)

# =================================
# # Exit definition
# =================================
def Exit(env, a, n):
    x = Job()
    while x.id < n:
        receive = a.receive()
        x = yield env.execute(receive)
    print("TH = %g" % ( x.id / env.now))

# =================================
# # GBME Model
# =================================
ta = 1.0
te = 1.0
n  = 1000
N = 10

env = Environment()
a = Channel(env)
b = Channel(env)
c = Channel(env)

G = env.process(Generator(env, a, lambda: random.exponential(ta)))
B = env.process(Buffer(env, a, b,  N))
M = env.process(Machine(env, b, c, lambda: random.exponential(te)))
E = env.process(Exit(env, c, n))

env.run()
print ("simulation has ended")

TH = 0.963514
simulation has ended


# D. Single Machine Model

In [22]:

# =================================
# # Job definition
# =================================
@dataclass
class Job:
    entrytime: float = 0.0

# =================================
# # Generator definition
# =================================
def Generator(env, a, u):
    while True:
        x = Job(entrytime = env.now)
        send = a.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)

# =================================
# # Buffer definition
# =================================
def Buffer(env, a, b):
    xs = []
    while True:
        receiving = a.receive()
        sending = b.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]       
        
# =================================
# # Machine definition
# =================================
def Machine(env, a, b, u):
    while True:
        receive = a.receive()
        x = yield env.execute(receive)
        
        send = b.send(x)
        yield env.execute(send)

        delay = u()
        yield env.timeout(delay)


# =================================
# # Exit definition
# =================================
def Exit(env, a, n):
    sumw = 0.0
    for i in range(n):
        receive = a.receive()
        x = yield env.execute(receive)
        sumw = sumw + (env.now - x.entrytime)
    print("Mean waiting time spent in buffer = %g" % (sumw / n))

# =================================
# # GBME Model
# =================================
n  = 100000

env = Environment()
a = Channel(env)
b = Channel(env)
c = Channel(env)

G = env.process(Generator(env, a, lambda: random.uniform(0.0, 2.0)))
B = env.process(Buffer(env, a, b))
M = env.process(Machine(env, b, c, lambda: random.uniform(0.0, 1.0)))
E = env.process(Exit(env, c, n))

env.run(until=E)
print ("simulation has ended")

Mean waiting time spent in buffer = 0.133427
simulation has ended


# E. Multi machine model

In [24]:

# =================================
# # Job definition
# =================================
@dataclass
class Job:
    entrytime: float = 0.0

# =================================
# # Generator definition
# =================================
def Generator(env, a, u):
    while True:
        x = Job(entrytime = env.now)
        send = a.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)

# =================================
# # Buffer definition
# =================================
def Buffer(env, a, b):
    xs = []
    while True:
        receiving = a.receive()
        sending = b.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]       
        
# =================================
# # Machine definition
# =================================
def Machine(env, a, b, u):
    while True:
        receive = a.receive()
        x = yield env.execute(receive)
        
        send = b.send(x)
        yield env.execute(send)

        delay = u()
        yield env.timeout(delay)


# =================================
# # Exit definition
# =================================
def Exit(env, a, n):
    sumw = 0.0
    for i in range(n):
        receive = a.receive()
        x = yield env.execute(receive)
        sumw = sumw + (env.now - x.entrytime)
    print("Mean waiting time spent in buffer = %g" % (sumw / n))

# =================================
# # GBME Model
# =================================
m = 2
n = 100000

env = Environment()
a = Channel(env)
b = Channel(env)
c = Channel(env)

G = env.process(Generator(env, a, lambda: random.exponential(0.111)))
B = env.process(Buffer(env, a, b))
Ms = [ env.process(Machine(env, b, c, lambda: random.uniform(0.0, 0.4))) for j in range(m) ]
E = env.process(Exit(env, c, n))

env.run(until=E)
print ("simulation has ended")

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
Mean waiting time spent in buffer = 0.569621
simulation has ended


# F. Serial production line

In [42]:
# =================================
# # Job definition
# =================================
@dataclass
class Job:
    entrytime: float = 0.0

# =================================
# # Generator definition
# =================================
def Generator(env, a, u):
    while True:
        x = Job(entrytime = env.now)
        send = a.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)

# =================================
# # Buffer definition
# =================================
def Buffer(env, a, b):
    xs = []
    while True:
        receiving = a.receive()
        sending = b.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]       
        
# =================================
# # Machine definition
# =================================
def Machine(env, a, b, u):
    while True:
        receive = a.receive()
        x = yield env.execute(receive)
        
        send = b.send(x)
        yield env.execute(send)

        delay = u()
        yield env.timeout(delay)


# =================================
# # Exit definition
# =================================
def Exit(env, a, n):
    sumw = 0.0
    for i in range(n):
        receive = a.receive()
        x = yield env.execute(receive)
        sumw = sumw + (env.now - x.entrytime)
    return sumw / n

# =================================
# # Mline Model
# =================================
def Model():
    n = 100000

    env = Environment()
    a = [ Channel(env) for j in range(3) ]
    b = [ Channel(env) for j in range(2) ]

    G = env.process(Generator(env, a[0], lambda: random.exponential(0.5)))
    B1 = env.process(Buffer(env, a[0], b[0]))
    M1 = env.process(Machine(env, b[0], a[1], lambda: 0.33))
    B1 = env.process(Buffer(env, a[1], b[1]))
    M1 = env.process(Machine(env, b[1], a[2], lambda: random.uniform(0.0, 0.8)))
    E = env.process(Exit(env, a[2], n))
    env.run(until=E)
    
    return E.value
    

# =================================
# # Experiment
# =================================
  
def Experiment():
    m = 10
    phi = 0.0
    sum1 = 0.0
    sum2 = 0.0
    smean = 0.0
    svar = 0.0
    
    for i in range(m):
        phi = Model()
        sum1 = sum1 + phi
        sum2 = sum2 + phi*phi
        print("Mean flow time in run %d is %g" %(i+1, phi))
        
    smean = sum1 / m
    svar = sum2 / m - smean * smean
    print("Mean flow time estimate is %g +- %g" % (smean, 1.96 * math.sqrt(svar/m) ))

# =================================
# # Main
# =================================
Experiment()
print ("simulation has ended")

Mean flow time in run 1 is 1.1395


KeyboardInterrupt: 

############################################################